In [1]:
import numpy as np
import torch
import torch.nn as nns
import jax
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
import jax.numpy as jnp
from jax import jit, lax, value_and_grad
import jax.random as random
import pandas as pd
import pylab as plt
from scipy.stats import boxcox
from sklearn.preprocessing import normalize
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import OneHotEncoder

## Importing Data and Data Transformation

In [2]:
from google.colab import drive
drive.mount('/content/drive')

from scipy.stats import boxcox
import numpy as np
df = pd.read_csv('/content/drive/My Drive/tr_eikon_eod_data.csv', index_col = 0, parse_dates = True).dropna()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
aapl = pd.DataFrame(df['AAPL.O'])
log_returns = np.log(aapl['AAPL.O'] / aapl['AAPL.O'].shift())
log_returns = pd.DataFrame(log_returns.dropna())
log_returns.rename(columns={"AAPL.O": "log_return"}, inplace=True)

bins = [-np.inf, -0.03, -0.02, -0.015, -0.01, -0.007, -0.005, -0.003, -0.002, -0.001, -0.0005, 0,
        0.0005, 0.001, 0.002, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, np.inf]

labels = ['01Very Strong Negative', '02Strong Negative', '03Moderate Negative', '04Moderately Weak Negative',
          '05Weak Negative', '06Slightly Weak Negative', '07Very Slightly Weak Negative', '08Extremely Weak Negative',
          '09Super Weak Negative', '10Super Slightly Weak Negative', '11Neutral', '12Super Slightly Weak Positive',
          '13Super Weak Positive', '14Extremely Weak Positive', '15Very Slightly Weak Positive', '16Slightly Weak Positive',
          '17Weak Positive', '18Moderately Weak Positive', '19Moderate Positive', '20Strong Positive', '21Very Strong Positive']

log_returns['category'] = pd.cut(log_returns['log_return'], bins=bins, labels=labels)
encoder = OneHotEncoder(sparse=False)
encoded_categories = encoder.fit_transform(log_returns['category'].values.reshape(-1, 1))
X = []
y = []
window_size =20
output_size = 1
for i in range(len(encoded_categories) - window_size - output_size + 1):
    X.append(encoded_categories[i:i + window_size])
    y.append(encoded_categories[i + window_size:i + window_size + output_size])
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False)
y_train = y_train.reshape(y_train.shape[0], y_train.shape[1]*y_train.shape[2])
y_test = y_test.reshape(y_test.shape[0], y_test.shape[1]*y_test.shape[2])

print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


model = Sequential()

model.add(LSTM(units=200, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=100, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=100))
model.add(Dropout(0.2))

model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

prediction = model.predict(X_test)
predicted_classes = np.argmax(prediction, axis=1)
actual_classes = np.argmax(y_test, axis=1)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Shape of y_train: (1693, 21)
Shape of y_test: (424, 21)
Epoch 1/50
14/14 [==============================] - 9s 99ms/step - loss: 3.0204 - accuracy: 0.0744 - val_loss: 3.0360 - val_accuracy: 0.0401
Epoch 2/50
14/14 [==============================] - 0s 14ms/step - loss: 2.9743 - accuracy: 0.0762 - val_loss: 3.0316 - val_accuracy: 0.0542
Epoch 3/50
14/14 [==============================] - 0s 13ms/step - loss: 2.9487 - accuracy: 0.0874 - val_loss: 3.0150 - val_accuracy: 0.0731
Epoch 4/50
14/14 [==============================] - 0s 12ms/step - loss: 2.9378 - accuracy: 0.0945 - val_loss: 3.0624 - val_accuracy: 0.0708
Epoch 5/50
14/14 [==============================] - 0s 12ms/step - loss: 2.9447 - accuracy: 0.0986 - val_loss: 3.0118 - val_accuracy: 0.0472
Epoch 6/50
14/14 [==============================] - 0s 12ms/step - loss: 2.9338 - accuracy: 0.0998 - val_loss: 3.0014 - val_accuracy: 0.0755
Epoch 7/50
14/14 [==============================] - 0s 12ms/step - loss: 2.9249 - accuracy: 0.1028

In [98]:
sum_first_eleven = prediction[:, :12].sum(axis=1)
sum_last_eleven = prediction[:, 12:].sum(axis=1)
transformed_predictions = np.column_stack((sum_first_eleven, sum_last_eleven))
predicted_class = np.argmax(transformed_predictions, axis = 1)
tmp = pd.DataFrame()
tmp['direction_predicted'] = predicted_class
tmp['actual_class'] = actual_classes
tmp['direction_actual'] = np.where(tmp['actual_class'] >= 12, 1, 0)
accuracy = np.sum(tmp['direction_predicted'] == tmp['direction_actual']) / len(tmp)

print(f"Prediction Accuracy for AAPL Stock with cumulative sepreation = {accuracy * 100}%")

Prediction Accuracy for AAPL Stock with cumulative sepreation = 52.358490566037744%


In [99]:
tmp = pd.DataFrame()
tmp['predicted_class'] = predicted_classes
tmp['actual_class'] = actual_classes
tmp['direction_actual'] = np.where(tmp['actual_class'] >= 12, 1, 0)
tmp['direction_predicted'] = np.where(tmp['predicted_class'] >= 12, 1, 0)
tmp, np.sum(tmp['direction_actual']) / len(tmp)

(     predicted_class  actual_class  direction_actual  direction_predicted
 0                  9            17                 1                    0
 1                 17            16                 1                    1
 2                 17             1                 0                    1
 3                 17             4                 0                    1
 4                 19             5                 0                    1
 ..               ...           ...               ...                  ...
 419                1             3                 0                    0
 420                1            18                 1                    0
 421                1             8                 0                    0
 422                1            17                 1                    0
 423               19             7                 0                    1
 
 [424 rows x 4 columns],
 0.49764150943396224)

## For GS.N


In [ ]:
gs = pd.DataFrame(df['GS.N'])
log_returns = np.log(gs['GS.N'] / gs['GS.N'].shift())
log_returns = pd.DataFrame(log_returns.dropna())
log_returns.rename(columns={"GS.N": "log_return"}, inplace=True)

bins = [-np.inf, -0.01, -0.001, 0, 0.001, 0.01, np.inf]
labels = ['1Strong Negative', '2Negative', '3Weak Negative', '4Weak Positive', '5Positive', '6Strong Positive']

log_returns['category'] = pd.cut(log_returns['log_return'], bins=bins, labels=labels)

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
encoded_categories = encoder.fit_transform(log_returns['category'].values.reshape(-1, 1))
X = []
y = []
window_size = 5
output_size = 1
for i in range(len(encoded_categories) - window_size - output_size + 1):
    X.append(encoded_categories[i:i + window_size])
    y.append(encoded_categories[i + window_size:i + window_size + output_size])
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False)
y_train = y_train.reshape(y_train.shape[0], y_train.shape[1]*y_train.shape[2])
y_test = y_test.reshape(y_test.shape[0], y_test.shape[1]*y_test.shape[2])

print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

model = Sequential()

model.add(LSTM(units=100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))

model.add(LSTM(units=100))

model.add(Dense(50, activation='relu'))

model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

prediction = model.predict(X_test)
predicted_classes = np.argmax(prediction, axis=1)
actual_classes = np.argmax(y_test, axis=1)

tmp = pd.DataFrame()
tmp['predicted_class'] = predicted_classes
tmp['actual_class'] = actual_classes
tmp['direction_actual'] = np.where(tmp['actual_class'] >= 3, 1, 0)
tmp['direction_predicted'] = np.where(tmp['predicted_class'] >= 3, 1, 0)
accuracy = np.sum(tmp['direction_predicted'] == tmp['direction_actual']) / len(tmp)
print(f"Prediction Accuracy for AAPL Stock with simple sepreation = {accuracy * 100}%")

sum_first_three = prediction[:, :3].sum(axis=1)
sum_last_three = prediction[:, 3:].sum(axis=1)
transformed_predictions = np.column_stack((sum_first_three, sum_last_three))
predicted_class = np.argmax(transformed_predictions, axis = 1)
tmp = pd.DataFrame()
tmp['direction_predicted'] = predicted_class
tmp['actual_class'] = actual_classes
tmp['direction_actual'] = np.where(tmp['actual_class'] >= 3, 1, 0)
accuracy = np.sum(tmp['direction_predicted'] == tmp['direction_actual']) / len(tmp)

print(f"Prediction Accuracy for AAPL Stock with cumulative sepreation = {accuracy * 100}%")

Shape of y_train: (1705, 6)
Shape of y_test: (427, 6)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/50
54/54 [==============================] - 5s 22ms/step - loss: 1.6195 - accuracy: 0.2475 - val_loss: 1.5647 - val_accuracy: 0.2623
Epoch 2/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5665 - accuracy: 0.2528 - val_loss: 1.5653 - val_accuracy: 0.2693
Epoch 3/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5612 - accuracy: 0.2639 - val_loss: 1.5571 - val_accuracy: 0.2810
Epoch 4/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5617 - accuracy: 0.2657 - val_loss: 1.5673 - val_accuracy: 0.2600
Epoch 5/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5567 - accuracy: 0.2680 - val_loss: 1.5719 - val_accuracy: 0.2506
Epoch 6/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5623 - accuracy: 0.2569 - val_loss: 1.5568 - val_accuracy: 0.3091
Epoch 7/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5563 - accuracy: 0.2762 - val_loss: 1.5576 - val_accuracy: 0.2810
Epoch 8/50
54/54 [=

## MSFT.O

In [ ]:
msft = pd.DataFrame(df['MSFT.O'])
log_returns = np.log(msft['MSFT.O'] / msft['MSFT.O'].shift())
log_returns = pd.DataFrame(log_returns.dropna())
log_returns.rename(columns={"MSFT.O": "log_return"}, inplace=True)

bins = [-np.inf, -0.01, -0.001, 0, 0.001, 0.01, np.inf]
labels = ['1Strong Negative', '2Negative', '3Weak Negative', '4Weak Positive', '5Positive', '6Strong Positive']

log_returns['category'] = pd.cut(log_returns['log_return'], bins=bins, labels=labels)
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
encoded_categories = encoder.fit_transform(log_returns['category'].values.reshape(-1, 1))
X = []
y = []
window_size = 5
output_size = 1
for i in range(len(encoded_categories) - window_size - output_size + 1):
    X.append(encoded_categories[i:i + window_size])
    y.append(encoded_categories[i + window_size:i + window_size + output_size])
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False)
y_train = y_train.reshape(y_train.shape[0], y_train.shape[1]*y_train.shape[2])
y_test = y_test.reshape(y_test.shape[0], y_test.shape[1]*y_test.shape[2])

print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

model = Sequential()

model.add(LSTM(units=100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))

model.add(LSTM(units=100))

model.add(Dense(50, activation='relu'))

model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

prediction = model.predict(X_test)
predicted_classes = np.argmax(prediction, axis=1)
actual_classes = np.argmax(y_test, axis=1)

tmp = pd.DataFrame()
tmp['predicted_class'] = predicted_classes
tmp['actual_class'] = actual_classes
tmp['direction_actual'] = np.where(tmp['actual_class'] >= 3, 1, 0)
tmp['direction_predicted'] = np.where(tmp['predicted_class'] >= 3, 1, 0)
accuracy = np.sum(tmp['direction_predicted'] == tmp['direction_actual']) / len(tmp)
print(f"Prediction Accuracy for AAPL Stock with simple sepreation = {accuracy * 100}%")

sum_first_three = prediction[:, :3].sum(axis=1)
sum_last_three = prediction[:, 3:].sum(axis=1)
transformed_predictions = np.column_stack((sum_first_three, sum_last_three))
predicted_class = np.argmax(transformed_predictions, axis = 1)
tmp = pd.DataFrame()
tmp['direction_predicted'] = predicted_class
tmp['actual_class'] = actual_classes
tmp['direction_actual'] = np.where(tmp['actual_class'] >= 3, 1, 0)
accuracy = np.sum(tmp['direction_predicted'] == tmp['direction_actual']) / len(tmp)

print(f"Prediction Accuracy for AAPL Stock with cumulative sepreation = {accuracy * 100}%")

Shape of y_train: (1705, 6)
Shape of y_test: (427, 6)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/50
54/54 [==============================] - 5s 21ms/step - loss: 1.6565 - accuracy: 0.2411 - val_loss: 1.6439 - val_accuracy: 0.2857
Epoch 2/50
54/54 [==============================] - 0s 7ms/step - loss: 1.6025 - accuracy: 0.2669 - val_loss: 1.6250 - val_accuracy: 0.2857
Epoch 3/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5998 - accuracy: 0.2616 - val_loss: 1.6340 - val_accuracy: 0.2998
Epoch 4/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5984 - accuracy: 0.2534 - val_loss: 1.6471 - val_accuracy: 0.2740
Epoch 5/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5982 - accuracy: 0.2587 - val_loss: 1.6207 - val_accuracy: 0.3021
Epoch 6/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5942 - accuracy: 0.2499 - val_loss: 1.6336 - val_accuracy: 0.2927
Epoch 7/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5954 - accuracy: 0.2845 - val_loss: 1.6527 - val_accuracy: 0.2857
Epoch 8/50
54/54 [=

## INTC.O

In [ ]:
intc = pd.DataFrame(df['INTC.O'])
log_returns = np.log(intc['INTC.O'] / intc['INTC.O'].shift())
log_returns = pd.DataFrame(log_returns.dropna())
log_returns.rename(columns={"INTC.O": "log_return"}, inplace=True)

bins = [-np.inf, -0.01, -0.001, 0, 0.001, 0.01, np.inf]
labels = ['1Strong Negative', '2Negative', '3Weak Negative', '4Weak Positive', '5Positive', '6Strong Positive']

log_returns['category'] = pd.cut(log_returns['log_return'], bins=bins, labels=labels)
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
encoded_categories = encoder.fit_transform(log_returns['category'].values.reshape(-1, 1))
X = []
y = []
window_size = 5
output_size = 1
for i in range(len(encoded_categories) - window_size - output_size + 1):
    X.append(encoded_categories[i:i + window_size])
    y.append(encoded_categories[i + window_size:i + window_size + output_size])
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False)
y_train = y_train.reshape(y_train.shape[0], y_train.shape[1]*y_train.shape[2])
y_test = y_test.reshape(y_test.shape[0], y_test.shape[1]*y_test.shape[2])

print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

model = Sequential()

model.add(LSTM(units=100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))

model.add(LSTM(units=100))

model.add(Dense(50, activation='relu'))

model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

prediction = model.predict(X_test)
predicted_classes = np.argmax(prediction, axis=1)
actual_classes = np.argmax(y_test, axis=1)

tmp = pd.DataFrame()
tmp['predicted_class'] = predicted_classes
tmp['actual_class'] = actual_classes
tmp['direction_actual'] = np.where(tmp['actual_class'] >= 3, 1, 0)
tmp['direction_predicted'] = np.where(tmp['predicted_class'] >= 3, 1, 0)
accuracy = np.sum(tmp['direction_predicted'] == tmp['direction_actual']) / len(tmp)
print(f"Prediction Accuracy for AAPL Stock with simple sepreation = {accuracy * 100}%")

sum_first_three = prediction[:, :3].sum(axis=1)
sum_last_three = prediction[:, 3:].sum(axis=1)
transformed_predictions = np.column_stack((sum_first_three, sum_last_three))
predicted_class = np.argmax(transformed_predictions, axis = 1)
tmp = pd.DataFrame()
tmp['direction_predicted'] = predicted_class
tmp['actual_class'] = actual_classes
tmp['direction_actual'] = np.where(tmp['actual_class'] >= 3, 1, 0)
accuracy = np.sum(tmp['direction_predicted'] == tmp['direction_actual']) / len(tmp)

print(f"Prediction Accuracy for AAPL Stock with cumulative sepreation = {accuracy * 100}%")

Shape of y_train: (1705, 6)
Shape of y_test: (427, 6)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/50
54/54 [==============================] - 4s 21ms/step - loss: 1.6603 - accuracy: 0.2387 - val_loss: 1.5931 - val_accuracy: 0.3232
Epoch 2/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5977 - accuracy: 0.2499 - val_loss: 1.5981 - val_accuracy: 0.3279
Epoch 3/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5952 - accuracy: 0.2633 - val_loss: 1.5919 - val_accuracy: 0.2951
Epoch 4/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5971 - accuracy: 0.2469 - val_loss: 1.5968 - val_accuracy: 0.2904
Epoch 5/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5898 - accuracy: 0.2504 - val_loss: 1.5782 - val_accuracy: 0.3162
Epoch 6/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5886 - accuracy: 0.2545 - val_loss: 1.5778 - val_accuracy: 0.2904
Epoch 7/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5887 - accuracy: 0.2475 - val_loss: 1.5788 - val_accuracy: 0.3068
Epoch 8/50
54/54 [=

## AMZN.O

In [ ]:
amzn = pd.DataFrame(df['AMZN.O'])
log_returns = np.log(amzn['AMZN.O'] / amzn['AMZN.O'].shift())
log_returns = pd.DataFrame(log_returns.dropna())
log_returns.rename(columns={"AMZN.O": "log_return"}, inplace=True)

bins = [-np.inf, -0.01, -0.001, 0, 0.001, 0.01, np.inf]
labels = ['1Strong Negative', '2Negative', '3Weak Negative', '4Weak Positive', '5Positive', '6Strong Positive']

log_returns['category'] = pd.cut(log_returns['log_return'], bins=bins, labels=labels)
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
encoded_categories = encoder.fit_transform(log_returns['category'].values.reshape(-1, 1))
X = []
y = []
window_size = 5
output_size = 1
for i in range(len(encoded_categories) - window_size - output_size + 1):
    X.append(encoded_categories[i:i + window_size])
    y.append(encoded_categories[i + window_size:i + window_size + output_size])
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False)
y_train = y_train.reshape(y_train.shape[0], y_train.shape[1]*y_train.shape[2])
y_test = y_test.reshape(y_test.shape[0], y_test.shape[1]*y_test.shape[2])

print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

model = Sequential()

model.add(LSTM(units=100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))

model.add(LSTM(units=100))

model.add(Dense(50, activation='relu'))

model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

prediction = model.predict(X_test)
predicted_classes = np.argmax(prediction, axis=1)
actual_classes = np.argmax(y_test, axis=1)

tmp = pd.DataFrame()
tmp['predicted_class'] = predicted_classes
tmp['actual_class'] = actual_classes
tmp['direction_actual'] = np.where(tmp['actual_class'] >= 3, 1, 0)
tmp['direction_predicted'] = np.where(tmp['predicted_class'] >= 3, 1, 0)
accuracy = np.sum(tmp['direction_predicted'] == tmp['direction_actual']) / len(tmp)
print(f"Prediction Accuracy for AAPL Stock with simple sepreation = {accuracy * 100}%")

sum_first_three = prediction[:, :3].sum(axis=1)
sum_last_three = prediction[:, 3:].sum(axis=1)
transformed_predictions = np.column_stack((sum_first_three, sum_last_three))
predicted_class = np.argmax(transformed_predictions, axis = 1)
tmp = pd.DataFrame()
tmp['direction_predicted'] = predicted_class
tmp['actual_class'] = actual_classes
tmp['direction_actual'] = np.where(tmp['actual_class'] >= 3, 1, 0)
accuracy = np.sum(tmp['direction_predicted'] == tmp['direction_actual']) / len(tmp)

print(f"Prediction Accuracy for AAPL Stock with cumulative sepreation = {accuracy * 100}%")

Shape of y_train: (1705, 6)
Shape of y_test: (427, 6)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/50
54/54 [==============================] - 6s 22ms/step - loss: 1.6365 - accuracy: 0.2751 - val_loss: 1.6001 - val_accuracy: 0.2365
Epoch 2/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5569 - accuracy: 0.2886 - val_loss: 1.6068 - val_accuracy: 0.2459
Epoch 3/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5462 - accuracy: 0.2944 - val_loss: 1.5905 - val_accuracy: 0.2904
Epoch 4/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5510 - accuracy: 0.2856 - val_loss: 1.6022 - val_accuracy: 0.2553
Epoch 5/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5504 - accuracy: 0.2903 - val_loss: 1.6219 - val_accuracy: 0.2389
Epoch 6/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5473 - accuracy: 0.2897 - val_loss: 1.6020 - val_accuracy: 0.2553
Epoch 7/50
54/54 [==============================] - 0s 7ms/step - loss: 1.5462 - accuracy: 0.2850 - val_loss: 1.6158 - val_accuracy: 0.2365
Epoch 8/50
54/54 [=

In [6]:
aapl = pd.DataFrame(df['AAPL.O'])

In [63]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Reshape

# Assuming you have loaded your data into a DataFrame named 'aapl'
time_series_data = aapl['AAPL.O'].values

def create_dataset(time_series_data, n_steps):
    X, y = [], []
    for i in range(len(time_series_data) - n_steps):
        end_ix = i + n_steps
        if end_ix + 10 < len(time_series_data):
            X.append(time_series_data[i:end_ix])
            y.append(1 if time_series_data[end_ix + 1] > time_series_data[end_ix] else 0)
    return np.array(X), np.array(y)

n_steps = 40

X, y = create_dataset(time_series_data, n_steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False)

# Normalize the input features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape input features for CNN input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


In [64]:
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=64, kernel_size=26, activation='relu', input_shape=(40, 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())

# Reshape output of CNN to match LSTM input shape
cnn_model.add(Reshape((-1, cnn_model.output_shape[1])))

# Define the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(1, cnn_model.output_shape[2])))  # Input shape: (batch_size, timesteps, features)

# Combine the models
combined_model = Sequential()
combined_model.add(cnn_model)
combined_model.add(lstm_model)
combined_model.add(Dense(1, activation='sigmoid'))

# Compile the model
combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the combined model
combined_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
42/42 [==============================] - 3s 17ms/step - loss: 0.6937 - accuracy: 0.5269 - val_loss: 0.6931 - val_accuracy: 0.4970
Epoch 2/10
42/42 [==============================] - 0s 6ms/step - loss: 0.6924 - accuracy: 0.5150 - val_loss: 0.7037 - val_accuracy: 0.4970
Epoch 3/10
42/42 [==============================] - 0s 6ms/step - loss: 0.6929 - accuracy: 0.5292 - val_loss: 0.6930 - val_accuracy: 0.4970
Epoch 4/10
42/42 [==============================] - 0s 6ms/step - loss: 0.6913 - accuracy: 0.5269 - val_loss: 0.6954 - val_accuracy: 0.4970
Epoch 5/10
42/42 [==============================] - 0s 6ms/step - loss: 0.6924 - accuracy: 0.5217 - val_loss: 0.6945 - val_accuracy: 0.4970
Epoch 6/10
42/42 [==============================] - 0s 6ms/step - loss: 0.6918 - accuracy: 0.5142 - val_loss: 0.6938 - val_accuracy: 0.4970
Epoch 7/10
42/42 [==============================] - 0s 6ms/step - loss: 0.6910 - accuracy: 0.5269 - val_loss: 0.6938 - val_accuracy: 0.4970
Epoch 8/10
42/42 [=